In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score
from interpret.blackbox import LimeTabular
from interpret import show

from utils import DataLoader
from interpret.glassbox import (LogisticRegression,
                                ClassificationTree, 
                                ExplainableBoostingClassifier)
from interpret import show
from sklearn.metrics import f1_score, accuracy_score
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score, f1_score
# from interpret.ext.blackbox import LimeTabular
from lime.lime_tabular import LimeTabularExplainer
import numpy as np
import shap




s:\Explainable AI\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv('data/healthcare-dataset-stroke-data.csv')
data.columns = data.columns.str.strip()
print(data.dtypes)
print(data.nunique())   
categorical_cols = ["gender", "ever_married", "work_type", "Residence_type", "smoking_status"]
data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)
data = data.drop(columns=['id'])

id                     int64
gender                object
age                  float64
hypertension           int64
heart_disease          int64
ever_married          object
work_type             object
Residence_type        object
avg_glucose_level    float64
bmi                  float64
smoking_status        object
stroke                 int64
dtype: object
id                   5110
gender                  3
age                   104
hypertension            2
heart_disease           2
ever_married            2
work_type               5
Residence_type          2
avg_glucose_level    3979
bmi                   418
smoking_status          4
stroke                  2
dtype: int64


In [3]:
# Assuming data is your DataFrame after one-hot encoding and dropping 'id'

# Fill missing bmi values
data['bmi'] = data['bmi'].fillna(data['bmi'].mean())

# Drop 'id' column if still present
data = data.drop(columns=['id'], errors='ignore')

# Split features and target
X = data.drop('stroke', axis=1)
y = data['stroke']

# Convert boolean columns to integers
bool_cols = X.select_dtypes(include=['bool']).columns
X[bool_cols] = X[bool_cols].astype(int)

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
print(X_train.shape)
print(X_test.shape)

(4088, 16)
(1022, 16)


In [6]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f"F1 Score {f1_score(y_test, y_pred, average='macro')}")
print(f"Accuracy {accuracy_score(y_test, y_pred)}")


F1 Score 0.4843592330978809
Accuracy 0.9393346379647749


[For details about dice-ml](https://pypi.org/project/dice-ml/)

In [9]:

import dice_ml
# Dataset
dice_data = dice_ml.Data(dataframe=pd.concat([X_train, y_train], axis=1),
                      
                         # For perturbation strategy
                         continuous_features=['age', 
                                              'avg_glucose_level',
                                              'bmi'], 
                         outcome_name='stroke')
dice_model = dice_ml.Model(model=model, backend='sklearn')


In [16]:
query_instance = X_test[y_test == 1].iloc[[0]]
query_instance

,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Male,gender_Other,ever_married_Yes,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
79,43.0,0,0,143.43,45.9,1,0,1,0,0,1,0,1,0,0,0


In [13]:
exp = dice_ml.Dice(dice_data, dice_model, method='random')

In [17]:
dice_exp = exp.generate_counterfactuals(
    query_instance,
    total_CFs=4,
    desired_class="opposite"
)

dice_exp.visualize_as_dataframe(show_only_changes=True)

  0%|          | 0/1 [00:00<?, ?it/s]s:\Explainable AI\myenv\Lib\site-packages\dice_ml\explainer_interfaces\dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  candidate_cfs.at[k, selected_features[k][0]] = random_instances.at[k, selected_features[k][0]]
s:\Explainable AI\myenv\Lib\site-packages\dice_ml\explainer_interfaces\dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  candidate_cfs.at[k, selected_features[k][0]] = random_instances.at[k, selected_features[k][0]]
s:\Explainable AI\myenv\Lib\site-packages\dice_ml\explainer_interfaces\dice_random.py:116: FutureWarning: Setting an item of incompatible dtype i

No Counterfactuals found for the given configuration, perhaps try with different parameters... ; total time taken: 00 min 00 sec


UserConfigValidationException: No counterfactuals found for any of the query points! Kindly check your configuration.

In [30]:
import pandas as pd
import dice_ml
from dice_ml import Dice
from sklearn.ensemble import RandomForestClassifier

In [36]:
import dice_ml
from dice_ml.utils import helpers # helper functions
from sklearn.model_selection import train_test_split

data = pd.read_csv('data/healthcare-dataset-stroke-data.csv')
data = data.drop(columns=['id'])
data.columns = data.columns.str.strip()
target = data['stroke']
data = pd.get_dummies(data.drop(columns=['stroke']), drop_first=True)
data['bmi'] = data['bmi'].fillna(data['bmi'].mean())
# Convert all boolean columns to int (True → 1, False → 0)
for col in data.columns:
    if data[col].dtype == bool:
        data[col] = data[col].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    data, target, test_size=0.2, random_state=42, stratify=target
)

In [38]:
print(data.dtypes)


age                               float64
hypertension                        int64
heart_disease                       int64
avg_glucose_level                 float64
bmi                               float64
gender_Male                         int64
gender_Other                        int64
ever_married_Yes                    int64
work_type_Never_worked              int64
work_type_Private                   int64
work_type_Self-employed             int64
work_type_children                  int64
Residence_type_Urban                int64
smoking_status_formerly smoked      int64
smoking_status_never smoked         int64
smoking_status_smokes               int64
dtype: object


In [39]:
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)

,n_estimators,200
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [40]:
d = dice_ml.Data(
    dataframe=pd.concat([X_train, y_train], axis=1),
    continuous_features=['age', 'avg_glucose_level', 'bmi'],
    outcome_name='stroke'
)


In [41]:
m = dice_ml.Model(model=rf, backend="sklearn")


In [42]:
exp = Dice(d, m, method="random")
query_instance = X_test.iloc[[0]]
dice_exp = exp.generate_counterfactuals(
    query_instance,
    total_CFs=4,
    desired_class=0 
)

  0%|          | 0/1 [00:00<?, ?it/s]s:\Explainable AI\myenv\Lib\site-packages\dice_ml\explainer_interfaces\dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  candidate_cfs.at[k, selected_features[k][0]] = random_instances.at[k, selected_features[k][0]]
s:\Explainable AI\myenv\Lib\site-packages\dice_ml\explainer_interfaces\dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  candidate_cfs.at[k, selected_features[k][0]] = random_instances.at[k, selected_features[k][0]]
s:\Explainable AI\myenv\Lib\site-packages\dice_ml\explainer_interfaces\dice_random.py:116: FutureWarning: Setting an item of incompatible dtype i

In [43]:
dice_exp.visualize_as_dataframe(show_only_changes=True)


Query instance (original outcome : 0)


,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Male,gender_Other,ever_married_Yes,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes,stroke
0,63.0,0,0,78.230003,34.799999,1,0,1,0,1,0,0,0,0,1,0,0



Diverse Counterfactual set (new outcome: 0)


,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Male,gender_Other,ever_married_Yes,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes,stroke
0,-,-,-,-,74.950911,-,-,-,-,-,-,-,-,-,-,-,-
1,-,-,-,-,46.879106,-,-,-,-,-,-,-,-,-,-,-,-
2,-,-,-,157.83,-,-,-,-,-,-,-,-,-,-,-,-,-
3,-,-,-,205.62,-,-,-,-,-,-,-,-,-,-,-,-,-
